In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

In [3]:
model_path  ='/CV/xhr_project/llm/model/Meta-Llama-3/LLM-Research/Meta-Llama-3-8B-Instruct'
# model_path2 = '/CV/xhr_project/llm/model/llama2/Llama-2-7b-chat'
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16).to("cuda:1")  # device_map="auto"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
def bulid_input(prompt, history=[]):
    history = []
    system_format='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{content}<|eot_id|>'
    user_format='<|start_header_id|>user<|end_header_id|>\n\n{content}<|eot_id|>'
    assistant_format='<|start_header_id|>assistant<|end_header_id|>\n\n{content}<|eot_id|>\n'
    history.append({'role':'user','content':prompt})
    prompt_str = ''
    # 拼接历史对话
    for item in history:
        if item['role']=='user':
            prompt_str+=user_format.format(content=item['content'])
        else:
            prompt_str+=assistant_format.format(content=item['content'])
    return prompt_str + '<|start_header_id|>assistant<|end_header_id|>\n\n'

In [8]:
def qa(prompt):
    input_str = bulid_input(prompt=prompt, history=[])
    # print(input_str)
    input_ids = tokenizer.encode(input_str, add_special_tokens=False, return_tensors='pt').cuda()

    outputs = model.generate(
        input_ids=input_ids, max_new_tokens=128, do_sample=True,
        top_p=0.9, temperature=0.5, repetition_penalty=1.1, eos_token_id=tokenizer.encode('<|eot_id|>')[0], pad_token_id=tokenizer.eos_token_id
        )
    outputs = outputs.tolist()[0][len(input_ids[0]):]
    response = tokenizer.decode(outputs)
    # print(response.strip().replace('<|eot_id|>', ""))
    response = response.strip().replace('<|eot_id|>', "").replace('<|start_header_id|>assistant<|end_header_id|>\n\n', '').replace('<|start_header_id|>assistant<|end_header_id|>\r\n\r\n', '').replace('assistant<|end_header_id|>\n\n', '').strip()

    return response


In [1]:
coin_template_few = (
                "Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?\nA: The coin was flipped by Ka and Sherrie. So the coin was flipped 2 times, which is an even number. The coinstarted heads up, so after an even number of flips, it will still be heads up. So the answer is yes.\n\n"
                "Q: A coin is heads up. Jamey fips the coin. Teressa flips the coin. Is the coin still heads up?\nA: The coin was flipped by Jamey and Teressa. So the coin was fipped 2 times, which is an even number. Thecoin started heads up, so after an even number of flips, it will still be heads up. So the answer is yes.\n\n"
                "O: A coin is heads up. Maybelle fips the coin. Shalonda does not fip the coin. Is the coin still heads up?\nA: The coin was flipped by Maybelle. So the coin was flipped 1 time, which is an odd number. The coin startedheads up, so after an odd number of flips, it will be tails up. So the answer is no.\n\n"
                "Q: {question}").strip()

coin_template_non = '{question}'

coin_template_zero = 'Take a deep breath and work on this problem step-by-step.\nQuestion: {question}'

In [2]:
print(coin_template_few.format(question='A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?'))

Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?
A: The coin was flipped by Ka and Sherrie. So the coin was flipped 2 times, which is an even number. The coinstarted heads up, so after an even number of flips, it will still be heads up. So the answer is yes.

Q: A coin is heads up. Jamey fips the coin. Teressa flips the coin. Is the coin still heads up?
A: The coin was flipped by Jamey and Teressa. So the coin was fipped 2 times, which is an even number. Thecoin started heads up, so after an even number of flips, it will still be heads up. So the answer is yes.

O: A coin is heads up. Maybelle fips the coin. Shalonda does not fip the coin. Is the coin still heads up?
A: The coin was flipped by Maybelle. So the coin was flipped 1 time, which is an odd number. The coin startedheads up, so after an odd number of flips, it will be tails up. So the answer is no.

Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still 

In [10]:
letter_template_few = (
                'Take the last letters of the words in "Elon Musk" and concatenate them.\nA: The last letter of "Elon" is "n". The last letter of "Musk" is "k". Concatenating them is "nk". The answer is nk.\n\n'
                'Q: Take the first letters of the words in "Larry Page" and concatenate them.\nA: The first letter of "Larry" is "L". The first letter of "Page" is "'P". Concatenating them is "LP". The answer is LP.\n\n'
                'O: Take the second letters of the words in "Sergey Brin" and concatenate them.\nA: The second letter of "Sergey" is "r". The second letter of "Brin" is "'i". Concatenating them is "ri". The answer is ri.\n\n'
                'Q: Take the {index} letters of the words in {question} and concatenate them.\nA:').strip()

letter_template_non = 'Q: Take the {index} letters of the words in {question} and concatenate them.\nA: '

letter_template_zero = ("Let's think step by step.\n"
                        "Q: Take the {index} letters of the words in '{question}' and concatenate them.\nA: ").strip()

In [11]:
prompt = letter_template_zero.format(index='second', question='Morgan Calderon')
print(qa(prompt))

Let's break it down step by step.

1. Take the first word "Morgan":
	* The second letter is "o".
2. Take the second word "Calderon":
	* The second letter is "a".

Now, let's concatenate these two letters:
"o" + "a" = "oa"

So, the answer is "oa".


In [12]:
import pandas as pd
from tqdm import tqdm
import random

df = pd.read_csv('dataset/full_name_ordinal.csv')
# filtered_df = df[df['length'] == 7]
# sampled_df7 = filtered_df.sample(n=100, random_state=1)
sampled_df = df.sample(n=200, random_state=1)

In [13]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['Content']
    index2 = row['ordinal_index']

    result = qa(prompt=letter_template_zero.format(index=index2, question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    results_non.append(result)



sampled_df.to_csv('output/result_letter_zero.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

200it [07:38,  2.29s/it]


In [168]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['Content']
    index2 = row['ordinal_index']

    result = qa(prompt=letter_template_few.format(index=index2, question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()

    results_non.append(result)


sampled_df.to_csv('output/result_letter_few.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [10:01,  3.01s/it]


In [169]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['Content']
    index2 = row['ordinal_index']

    result = qa(prompt=letter_template_non.format(index=index2, question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()

    results_non.append(result)



sampled_df.to_csv('output/result_letter_non.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [03:28,  1.04s/it]


In [110]:
sampled_df = df.sample(n=200, random_state=1)
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['inputs']

    result = qa(prompt=coin_template_non.format(question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    results_non.append(result)



sampled_df.to_csv('output/result_coin_non.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [12:04,  3.62s/it]


In [132]:
def replace_with_ordinal(index):
    ordinal_map = {0: "first", 1: "second", 2: "third", 3: "fourth", 4: "fifth", 5: "sixth", 6: "seventh", 7: "eighth", 8: "ninth", 9: "tenth", -1: "last"}  # 添加更多的序数词，根据需要
    ordinal_index = ordinal_map.get(index, str(index))
    return ordinal_index

In [143]:
import pandas as pd
from tqdm import tqdm
import random

df = pd.read_csv('dataset/full_name.csv')
results = []
for index, row in tqdm(df.iterrows()):
    index2 = row['num']
    
    ordinal_index = replace_with_ordinal(index2)
    # print(f"index: {index} ---convert to ordinal: {ordinal_index}")
    # print()
    df.at[index, 'ordinal_index'] = ordinal_index
    # print(result)
    # results_non.append(result)
columns = list(df.columns)
columns.insert(2, columns.pop(columns.index('ordinal_index')))
df = df.reindex(columns=columns)
df.to_csv('output/full_name_ordinal.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

0it [00:00, ?it/s]

1000it [00:00, 26878.98it/s]


# 分割GSM8K

In [53]:
import json
content = []
with open('/CV/xhr_project/llm/Paper/nips/dataset/grade-school-math-master/grade_school_math/data/test.jsonl', 'r')as file:
    for line in file:
        data = json.loads(line)
        content.append(data)

In [54]:
import random
random.seed(11)
test_set = random.sample(content, 100)

In [55]:
GSM8K_template_non = '{question}'

GSM8K_template_zero = "Let's think step by step. {question}"

In [56]:
prompt = GSM8K_template_zero.format(question='A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?')
print(prompt, qa(prompt))

Let's think step by step. A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take? Let's break it down step by step.

A robe takes:

* 2 bolts of blue fiber
* Half as much white fiber as blue fiber

Since there are 2 bolts of blue fiber, half as much would be:

* 1 bolt of white fiber (since 2/2 = 1)

So, the robe takes a total of:

* 2 bolts of blue fiber
* 1 bolt of white fiber

In total, the robe takes 3 bolts of fiber: 2 blue and 1 white.


In [57]:
from tqdm import tqdm
for item in tqdm(test_set):
    prompt = GSM8K_template_zero.format(question=item['question'])
    result = qa(prompt)
    item['pred'] = result
    item['ground truth'] = item.pop('answer')


100%|██████████| 100/100 [05:17<00:00,  3.17s/it]


In [58]:
test_set[:5]

[{'question': "Colby loves going to the movies and every month his parents give him $150 to spend at the movies. Tickets for Fridays and Saturdays cost $10. Tickets for any other day cost $7. Popcorn costs $8 and boxes of candy cost $2. It is the last day of the month and it's a Friday. He wants to make sure he gets a popcorn and box of candy that night. How many movies can he see if he already saw 5 movies on a Friday or Saturday, 8 movies on other days, had 2 tubs of popcorn, and four boxes of candy that month?",
  'pred': "Let's break this down step by step.\n\nColby has already spent money on:\n\n* 5 movies on Fridays or Saturdays (cost: 5 x $10 = $50)\n* 8 movies on other days (cost: 8 x $7 = $56)\n* 2 tubs of popcorn (cost: 2 x $8 = $16)\n* 4 boxes of candy (cost: 4 x $2 = $8)\n\nTotal amount spent so far: $50 + $56 + $16 + $8 = $130\n\nColby still has $150 - $130 = $20 left to",
  'ground truth': 'He has spent $50 on Fri/Sat movies because 5 x 10 = <<50=50>>50\nHe has spent $56 

In [59]:
import json

# 指定要写入的文件名
output_file = "output/GSM8K_zero.jsonl"

# 打开文件以写入模式
with open(output_file, 'w') as f:
    # 遍历 JSON 数据列表
    for json_data in test_set:
        # 将每个 JSON 对象转换为字符串，并添加换行符
        json_line = json.dumps(json_data) + '\n'
        # 将 JSON 对象写入文件
        f.write(json_line)

print("JSONL 文件已创建：", output_file)

JSONL 文件已创建： output/GSM8K_zero.jsonl


# 分割strategyQA

In [60]:
def fine_targe(data):
# 使用列表推导式查找值为1的键
    keys_with_value_1 = [key for key, value in data.items() if value == 1]
    return ''.join(keys_with_value_1)


In [61]:
Strategy_template_non = '{question}'

Strategy_template_zero = "Let's think step by step. {question}"

In [62]:
import json
import random
random.seed(11)
# content = []
with open('/CV/xhr_project/llm/Paper/nips/dataset/StrategyQA.json', 'r')as file:
    data = json.load(file)
    content = data['examples']
    content = random.sample(content, 100)

In [63]:
content

[{'input': 'Does a person suffering from Thalassophobia enjoy oceanography?',
  'target_scores': {'Yes': 0, 'No': 1},
  'target': 'No. Thalassophobia is a deep and persistent fear of the sea. Oceanography is the study of bodies of water. Oceanographers frequently observe and interact with bodies of water such as lakes, seas, and oceans.'},
 {'input': 'Would most grand masters know what the French Defense is?',
  'target_scores': {'Yes': 1, 'No': 0},
  'target': 'Yes. Grand master is the highest title a chess player can get. The French Defense is a well known chess opening that is in many books.'},
 {'input': 'Did goddess Friday is named after despise felines?',
  'target_scores': {'Yes': 0, 'No': 1},
  'target': 'No. Felines are a species of animals that include lions, tigers, and domestic cats. Friday is named after the Norse goddess Freya.  Freya is often depicted in art with cats. Freya had two cats that pulled her magical chariot.'},
 {'input': 'Has Don King killed more people than

In [64]:
result = []
for item in tqdm(content):
    # print(item.keys())
    question = item['input']
    item['ground_label'] = fine_targe(item.pop('target_scores'))
    item['ground_truth'] = item.pop('target')
    prompt = Strategy_template_zero.format(question=item['input'])
    pred = qa(prompt)
    item['pred'] = pred
    result.append(item)


100%|██████████| 100/100 [05:09<00:00,  3.09s/it]


In [65]:
result

[{'input': 'Does a person suffering from Thalassophobia enjoy oceanography?',
  'ground_label': 'No',
  'ground_truth': 'No. Thalassophobia is a deep and persistent fear of the sea. Oceanography is the study of bodies of water. Oceanographers frequently observe and interact with bodies of water such as lakes, seas, and oceans.',
  'pred': "That's an interesting question!\n\nI would say that it's unlikely for someone with Thalassophobia to enjoy oceanography, as the fear of the ocean (Thalassophobia) is often accompanied by feelings of anxiety or discomfort when thinking about or being near the ocean.\n\nOceanography, on the other hand, involves studying the ocean and its phenomena, which might require being in close proximity to the ocean or even engaging in activities like scuba diving or boat travel.\n\nGiven the conflicting nature of these two concepts, I'd argue that a person with Thalassophobia would likely find it challenging to enjoy oceanography due to their underlying fear"},


In [66]:
import json

# 指定要写入的文件名
output_file = "output/Strategy_zero.jsonl"

# 打开文件以写入模式
with open(output_file, 'w') as f:
    # 遍历 JSON 数据列表
    for json_data in result:
        # 将每个 JSON 对象转换为字符串，并添加换行符
        json_line = json.dumps(json_data) + '\n'
        # 将 JSON 对象写入文件
        f.write(json_line)

print("JSONL 文件已创建：", output_file)

JSONL 文件已创建： output/Strategy_zero.jsonl
